# 📊 Análisis Exploratorio de Datos (EDA) y Transformación (ETL)
## Proyecto: Análisis de Exportaciones Colombianas

---

**Objetivo:** Realizar un análisis exploratorio completo del dataset de exportaciones y preparar los datos para modelos de Machine Learning.

**Dataset:** DATAPROYECTO.xlsx - Contiene información detallada sobre exportaciones colombianas

**Contenido:**
1. Carga y exploración inicial
2. Análisis de valores faltantes
3. Análisis estadístico descriptivo
4. Visualizaciones exploratorias
5. Detección de outliers
6. Transformación y limpieza (ETL)
7. Feature Engineering
8. Exportación de datos limpios

## 1. Importación de Librerías

In [ ]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Librerías para análisis estadístico
from scipy import stats
from scipy.stats import skew, kurtosis

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas exitosamente")

## 2. Carga de Datos

In [ ]:
# Cargar el dataset desde Excel
df = pd.read_excel('DATAPROYECTO.xlsx', sheet_name='Detalle')

# Crear una copia para trabajo
df_original = df.copy()

print(f"📦 Dataset cargado exitosamente")
print(f"   Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
print(f"   Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 3. Exploración Inicial del Dataset

In [ ]:
# Vista general de las primeras filas
print("📋 PRIMERAS 10 FILAS DEL DATASET")
print("="*100)
display(df.head(10))

In [ ]:
# Información general del dataset
print("ℹ️ INFORMACIÓN GENERAL DEL DATASET")
print("="*100)
df.info()

In [ ]:
# Nombres de las columnas
print("📝 COLUMNAS DEL DATASET:")
print("="*100)
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# Tipos de datos por columna
print("🔢 TIPOS DE DATOS:")
print("="*100)
tipos = df.dtypes.value_counts()
print(tipos)
print(f"\n📊 Resumen:")
print(f"   • Numéricas (int64): {(df.dtypes == 'int64').sum()}")
print(f"   • Numéricas (float64): {(df.dtypes == 'float64').sum()}")
print(f"   • Categóricas (object): {(df.dtypes == 'object').sum()}")

## 4. Análisis de Valores Faltantes

In [ ]:
# Calcular valores nulos
missing_data = pd.DataFrame({
    'Columna': df.columns,
    'Valores_Nulos': df.isnull().sum(),
    'Porcentaje_%': (df.isnull().sum() / len(df)) * 100
}).sort_values('Valores_Nulos', ascending=False)

missing_data = missing_data[missing_data['Valores_Nulos'] > 0]

print("❌ ANÁLISIS DE VALORES FALTANTES:")
print("="*100)
if len(missing_data) > 0:
    display(missing_data)
    print(f"\n⚠️ Total de columnas con valores faltantes: {len(missing_data)}")
else:
    print("✅ No hay valores faltantes en el dataset")

In [ ]:
# Visualización de valores faltantes
if len(missing_data) > 0:
    plt.figure(figsize=(12, 6))
    plt.barh(missing_data['Columna'], missing_data['Porcentaje_%'], color='coral')
    plt.xlabel('Porcentaje de Valores Faltantes (%)', fontsize=12)
    plt.ylabel('Columnas', fontsize=12)
    plt.title('Valores Faltantes por Columna', fontsize=14, fontweight='bold')
    plt.grid(axis='x', alpha=0.3)
    for i, v in enumerate(missing_data['Porcentaje_%']):
        plt.text(v + 0.5, i, f'{v:.2f}%', va='center')
    plt.tight_layout()
    plt.show()

## 5. Estadísticas Descriptivas

In [ ]:
# Estadísticas para variables numéricas
print("📈 ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS:")
print("="*100)
display(df.describe().T)

In [ ]:
# Estadísticas para variables categóricas
print("📊 ESTADÍSTICAS DESCRIPTIVAS - VARIABLES CATEGÓRICAS:")
print("="*100)
categorical_cols = df.select_dtypes(include=['object']).columns

cat_stats = pd.DataFrame({
    'Columna': categorical_cols,
    'Valores_Únicos': [df[col].nunique() for col in categorical_cols],
    'Valor_Más_Frecuente': [df[col].mode()[0] if len(df[col].mode()) > 0 else None for col in categorical_cols],
    'Frecuencia_Max': [df[col].value_counts().iloc[0] for col in categorical_cols]
})
display(cat_stats)

## 6. Análisis de Distribuciones - Variables Numéricas Clave

In [ ]:
# Seleccionar las variables numéricas más importantes para visualización
numerical_cols_key = ['Valor FOB (USD)', 'Peso en kilos netos', 'Peso en kilos brutos', 
                      'Cantidad(es)', 'Precio Unitario FOB (USD) Peso Neto']

# Crear visualizaciones de distribución
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols_key):
    # Histograma con KDE
    sns.histplot(df[col].dropna(), kde=True, ax=axes[idx], color='steelblue')
    axes[idx].set_title(f'Distribución: {col}', fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frecuencia')
    
    # Añadir estadísticas en el gráfico
    mean_val = df[col].mean()
    median_val = df[col].median()
    axes[idx].axvline(mean_val, color='red', linestyle='--', label=f'Media: {mean_val:.2f}')
    axes[idx].axvline(median_val, color='green', linestyle='--', label=f'Mediana: {median_val:.2f}')
    axes[idx].legend()

plt.tight_layout()
plt.show()

print("📊 Las distribuciones muestran el comportamiento de las variables numéricas principales")

In [ ]:
# Box plots para detectar outliers
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols_key):
    sns.boxplot(y=df[col].dropna(), ax=axes[idx], color='lightcoral')
    axes[idx].set_title(f'Box Plot: {col}', fontweight='bold')
    axes[idx].set_ylabel(col)

plt.tight_layout()
plt.show()

print("📦 Los box plots permiten identificar valores atípicos en cada variable")

## 7. Análisis de Variables Categóricas

In [ ]:
# Top 10 países de destino
print("🌍 TOP 10 PAÍSES DE DESTINO:")
print("="*100)
top_paises = df['País de Destino'].value_counts().head(10)
print(top_paises)

plt.figure(figsize=(12, 6))
top_paises.plot(kind='barh', color='skyblue')
plt.title('Top 10 Países de Destino', fontsize=14, fontweight='bold')
plt.xlabel('Número de Exportaciones')
plt.ylabel('País')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Distribución por continente
print("🌎 DISTRIBUCIÓN POR CONTINENTE:")
print("="*100)
continente_dist = df['Continente Destino'].value_counts()
print(continente_dist)

plt.figure(figsize=(10, 6))
plt.pie(continente_dist.values, labels=continente_dist.index, autopct='%1.1f%%', 
        startangle=90, colors=sns.color_palette('pastel'))
plt.title('Distribución de Exportaciones por Continente', fontsize=14, fontweight='bold')
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
# Top 10 departamentos de origen
print("📍 TOP 10 DEPARTAMENTOS DE ORIGEN:")
print("="*100)
top_deptos = df['Departamento Origen'].value_counts().head(10)
print(top_deptos)

plt.figure(figsize=(12, 6))
top_deptos.plot(kind='bar', color='lightgreen')
plt.title('Top 10 Departamentos de Origen', fontsize=14, fontweight='bold')
plt.xlabel('Departamento')
plt.ylabel('Número de Exportaciones')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Análisis de vías de transporte
print("🚢 VÍAS DE TRANSPORTE:")
print("="*100)
transporte_dist = df['Vía de transporte'].value_counts()
print(transporte_dist)

plt.figure(figsize=(10, 6))
transporte_dist.plot(kind='bar', color='coral')
plt.title('Distribución por Vía de Transporte', fontsize=14, fontweight='bold')
plt.xlabel('Vía de Transporte')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Análisis de Correlaciones

In [ ]:
# Matriz de correlación para variables numéricas
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Calcular matriz de correlación
correlation_matrix = df[numerical_cols].corr()

# Visualizar matriz de correlación
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación - Variables Numéricas', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🔗 CORRELACIONES MÁS FUERTES (|r| > 0.7):")
print("="*100)
# Encontrar correlaciones fuertes
high_corr = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

if high_corr:
    for var1, var2, corr in high_corr:
        print(f"   • {var1} ↔ {var2}: r = {corr:.3f}")
else:
    print("   No se encontraron correlaciones muy fuertes (|r| > 0.7)")

## 9. Detección de Outliers

In [ ]:
# Función para detectar outliers usando IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Analizar outliers en variables clave
print("🎯 ANÁLISIS DE OUTLIERS (Método IQR):")
print("="*100)

outlier_summary = []
for col in numerical_cols_key:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    pct_outliers = (len(outliers) / len(df)) * 100
    outlier_summary.append({
        'Variable': col,
        'Outliers': len(outliers),
        'Porcentaje_%': pct_outliers,
        'Límite_Inferior': lower,
        'Límite_Superior': upper
    })

outlier_df = pd.DataFrame(outlier_summary)
display(outlier_df)

## 10. ETL - Transformación y Limpieza de Datos

In [ ]:
# Crear copia para transformación
df_clean = df.copy()

print("🔧 INICIANDO PROCESO ETL...")
print("="*100)

# 1. Manejo de valores faltantes
print("\n1️⃣ Tratamiento de valores faltantes:")

# Para columnas numéricas: imputar con la mediana
numeric_cols_with_nulls = df_clean.select_dtypes(include=[np.number]).columns[df_clean.select_dtypes(include=[np.number]).isnull().any()].tolist()
for col in numeric_cols_with_nulls:
    median_val = df_clean[col].median()
    df_clean[col].fillna(median_val, inplace=True)
    print(f"   ✓ {col}: Imputado con mediana = {median_val:.2f}")

# Para columnas categóricas: imputar con 'Desconocido'
categorical_cols_with_nulls = df_clean.select_dtypes(include=['object']).columns[df_clean.select_dtypes(include=['object']).isnull().any()].tolist()
for col in categorical_cols_with_nulls:
    df_clean[col].fillna('Desconocido', inplace=True)
    print(f"   ✓ {col}: Imputado con 'Desconocido'")

print(f"\n   Total valores nulos restantes: {df_clean.isnull().sum().sum()}")

In [ ]:
# 2. Eliminación de duplicados
print("\n2️⃣ Eliminación de duplicados:")
duplicates_before = df_clean.duplicated().sum()
df_clean.drop_duplicates(inplace=True)
duplicates_after = df_clean.duplicated().sum()
print(f"   ✓ Duplicados eliminados: {duplicates_before}")
print(f"   ✓ Filas restantes: {len(df_clean):,}")

In [ ]:
# 3. Normalización de texto en columnas categóricas
print("\n3️⃣ Normalización de texto:")
text_columns = ['País de Destino', 'Departamento Origen', 'Continente Destino', 
                'Vía de transporte', 'Razón social actual Exportador']

for col in text_columns:
    if col in df_clean.columns:
        # Convertir a mayúsculas y eliminar espacios extras
        df_clean[col] = df_clean[col].str.upper().str.strip()
        print(f"   ✓ {col}: Normalizado")

In [ ]:
# 4. Creación de variables derivadas
print("\n4️⃣ Feature Engineering - Creación de nuevas variables:")

# Ratio Peso Bruto/Neto
df_clean['Ratio_Peso_Bruto_Neto'] = df_clean['Peso en kilos brutos'] / (df_clean['Peso en kilos netos'] + 1e-10)
print("   ✓ Ratio_Peso_Bruto_Neto: Creada")

# Valor por kilogramo
df_clean['Valor_Por_Kg'] = df_clean['Valor FOB (USD)'] / (df_clean['Peso en kilos netos'] + 1e-10)
print("   ✓ Valor_Por_Kg: Creada")

# Clasificación de valor de exportación
def clasificar_valor(valor):
    if valor < 1000:
        return 'Bajo'
    elif valor < 10000:
        return 'Medio'
    elif valor < 100000:
        return 'Alto'
    else:
        return 'Muy Alto'

df_clean['Categoria_Valor'] = df_clean['Valor FOB (USD)'].apply(clasificar_valor)
print("   ✓ Categoria_Valor: Creada (Bajo/Medio/Alto/Muy Alto)")

# Eficiencia de transporte (Valor por artículo)
df_clean['Valor_Por_Articulo'] = df_clean['Valor FOB (USD)'] / (df_clean['Número de artículos'] + 1e-10)
print("   ✓ Valor_Por_Articulo: Creada")

# Peso promedio por artículo
df_clean['Peso_Promedio_Articulo'] = df_clean['Peso en kilos netos'] / (df_clean['Número de artículos'] + 1e-10)
print("   ✓ Peso_Promedio_Articulo: Creada")

print(f"\n   📊 Total de columnas después de Feature Engineering: {len(df_clean.columns)}")

In [ ]:
# 5. Manejo de outliers extremos (opcional - usar con cuidado)
print("\n5️⃣ Manejo de outliers extremos:")
print("   ℹ️ Se aplicará winsorización para valores extremos (percentil 1% y 99%)")

# Aplicar winsorización a las variables numéricas clave
from scipy.stats.mstats import winsorize

cols_to_winsorize = ['Valor FOB (USD)', 'Peso en kilos netos', 'Cantidad(es)', 'Valor_Por_Kg']

for col in cols_to_winsorize:
    if col in df_clean.columns:
        # Winsorizar entre el 1% y 99%
        df_clean[col + '_winsorized'] = winsorize(df_clean[col], limits=[0.01, 0.01])
        print(f"   ✓ {col}: Winsorizado (1%-99%)")

## 11. Codificación de Variables Categóricas

In [ ]:
print("🔢 CODIFICACIÓN DE VARIABLES CATEGÓRICAS:")
print("="*100)

from sklearn.preprocessing import LabelEncoder

# Crear copia para codificación
df_encoded = df_clean.copy()

# Seleccionar columnas categóricas principales para codificar
categorical_to_encode = ['País de Destino', 'Continente Destino', 'Departamento Origen',
                         'Vía de transporte', 'Moneda de negociación', 'Forma de pago',
                         'Aduana De Embarque', 'Categoria_Valor']

label_encoders = {}

for col in categorical_to_encode:
    if col in df_encoded.columns:
        le = LabelEncoder()
        df_encoded[col + '_encoded'] = le.fit_transform(df_encoded[col].astype(str))
        label_encoders[col] = le
        print(f"   ✓ {col}: Codificado ({df_encoded[col].nunique()} categorías)")

print(f"\n   Total de variables codificadas: {len(label_encoders)}")

## 12. Escalamiento de Variables Numéricas

In [ ]:
print("📏 ESCALAMIENTO DE VARIABLES NUMÉRICAS:")
print("="*100)

from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Variables numéricas a escalar
numeric_cols_to_scale = ['Valor FOB (USD)', 'Peso en kilos netos', 'Peso en kilos brutos',
                         'Cantidad(es)', 'Precio Unitario FOB (USD) Peso Neto',
                         'Ratio_Peso_Bruto_Neto', 'Valor_Por_Kg', 'Valor_Por_Articulo']

# StandardScaler (Z-score normalization)
scaler_standard = StandardScaler()
for col in numeric_cols_to_scale:
    if col in df_encoded.columns:
        df_encoded[col + '_scaled'] = scaler_standard.fit_transform(df_encoded[[col]])

print("   ✓ StandardScaler aplicado (media=0, std=1)")

# MinMaxScaler (normalización 0-1)
scaler_minmax = MinMaxScaler()
for col in numeric_cols_to_scale:
    if col in df_encoded.columns:
        df_encoded[col + '_normalized'] = scaler_minmax.fit_transform(df_encoded[[col]])

print("   ✓ MinMaxScaler aplicado (rango 0-1)")
print(f"\n   Total de columnas después del escalamiento: {len(df_encoded.columns)}")

## 13. Validación de Datos Limpios

In [ ]:
print("✅ VALIDACIÓN DE DATOS LIMPIOS:")
print("="*100)

print(f"\n📊 Dimensiones finales:")
print(f"   • Filas: {df_encoded.shape[0]:,}")
print(f"   • Columnas: {df_encoded.shape[1]}")

print(f"\n❌ Valores nulos:")
nulls = df_encoded.isnull().sum().sum()
print(f"   • Total: {nulls}")

print(f"\n🔢 Tipos de datos:")
print(f"   • Numéricas: {len(df_encoded.select_dtypes(include=[np.number]).columns)}")
print(f"   • Categóricas: {len(df_encoded.select_dtypes(include=['object']).columns)}")

print(f"\n💾 Tamaño en memoria: {df_encoded.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n" + "="*100)
print("✅ PROCESO ETL COMPLETADO EXITOSAMENTE")
print("="*100)

## 14. Exportación de Datos Procesados

In [ ]:
print("💾 EXPORTANDO DATASETS PROCESADOS...")
print("="*100)

# Guardar diferentes versiones del dataset

# 1. Dataset limpio sin codificar (para análisis)
df_clean.to_csv('data_clean.csv', index=False)
print("   ✓ data_clean.csv guardado")

# 2. Dataset codificado y escalado (para ML)
df_encoded.to_csv('data_processed.csv', index=False)
print("   ✓ data_processed.csv guardado")

# 3. Dataset solo con variables numéricas (para clustering)
df_numeric = df_encoded.select_dtypes(include=[np.number])
df_numeric.to_csv('data_numeric.csv', index=False)
print("   ✓ data_numeric.csv guardado")

# 4. Guardar también en formato pickle para preservar tipos de datos
import pickle

with open('data_processed.pkl', 'wb') as f:
    pickle.dump(df_encoded, f)
print("   ✓ data_processed.pkl guardado")

# 5. Guardar los encoders y scalers
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)
print("   ✓ label_encoders.pkl guardado")

print("\n" + "="*100)
print("✅ TODOS LOS ARCHIVOS GUARDADOS EXITOSAMENTE")
print("="*100)

## 15. Resumen Final del EDA y ETL

In [ ]:
print("\n" + "="*100)
print("📋 RESUMEN EJECUTIVO - EDA Y ETL")
print("="*100)

print("\n🔍 DATOS ORIGINALES:")
print(f"   • Registros: {df_original.shape[0]:,}")
print(f"   • Variables: {df_original.shape[1]}")
print(f"   • Valores nulos: {df_original.isnull().sum().sum():,}")

print("\n🛠️ TRANSFORMACIONES APLICADAS:")
print("   1. ✓ Imputación de valores faltantes")
print("   2. ✓ Eliminación de duplicados")
print("   3. ✓ Normalización de texto")
print("   4. ✓ Feature Engineering (5 nuevas variables)")
print("   5. ✓ Tratamiento de outliers (winsorización)")
print("   6. ✓ Codificación de variables categóricas")
print("   7. ✓ Escalamiento de variables numéricas")

print("\n📊 DATOS PROCESADOS:")
print(f"   • Registros finales: {df_encoded.shape[0]:,}")
print(f"   • Variables finales: {df_encoded.shape[1]}")
print(f"   • Nuevas variables creadas: {df_encoded.shape[1] - df_original.shape[1]}")
print(f"   • Valores nulos: {df_encoded.isnull().sum().sum()}")

print("\n💾 ARCHIVOS GENERADOS:")
print("   • data_clean.csv - Dataset limpio sin codificar")
print("   • data_processed.csv - Dataset completo procesado")
print("   • data_numeric.csv - Solo variables numéricas")
print("   • data_processed.pkl - Dataset en formato pickle")
print("   • label_encoders.pkl - Encoders guardados")

print("\n🎯 INSIGHTS PRINCIPALES:")
print(f"   • Principal país destino: {df_clean['País de Destino'].mode()[0]}")
print(f"   • Principal continente: {df_clean['Continente Destino'].mode()[0]}")
print(f"   • Departamento con más exportaciones: {df_clean['Departamento Origen'].mode()[0]}")
print(f"   • Vía de transporte más usada: {df_clean['Vía de transporte'].mode()[0]}")
print(f"   • Valor FOB promedio: ${df_clean['Valor FOB (USD)'].mean():,.2f} USD")
print(f"   • Valor FOB mediano: ${df_clean['Valor FOB (USD)'].median():,.2f} USD")

print("\n" + "="*100)
print("🎉 ANÁLISIS EXPLORATORIO Y ETL FINALIZADOS")
print("📌 Los datos están listos para ser utilizados en modelos de Machine Learning")
print("="*100)

---

## 🎓 Conclusiones y Próximos Pasos

### Hallazgos Principales del EDA:

1. **Calidad de Datos**: El dataset presentaba valores faltantes moderados que fueron tratados adecuadamente
2. **Distribuciones**: Las variables numéricas muestran distribuciones asimétricas con presencia de outliers
3. **Correlaciones**: Se identificaron correlaciones esperadas entre peso bruto/neto y valores FOB
4. **Categorías Dominantes**: Concentración en ciertos países, departamentos y vías de transporte

### Transformaciones ETL Realizadas:

- ✅ Limpieza y normalización de datos
- ✅ Creación de variables derivadas relevantes
- ✅ Codificación de variables categóricas
- ✅ Escalamiento de variables numéricas
- ✅ Preparación de múltiples versiones del dataset

### Datasets Disponibles:

1. **data_clean.csv**: Para análisis exploratorios adicionales
2. **data_processed.csv**: Para modelos supervisados de ML
3. **data_numeric.csv**: Para modelos de clustering
4. **data_processed.pkl**: Para mantener tipos de datos Python

### Próximos Pasos:

1. **Regresión Lineal**: Predecir Valor FOB en función de variables continuas
2. **Regresión Logística**: Clasificar categorías de valor de exportación
3. **KNN**: Clasificación basada en similitud
4. **SVM**: Clasificación con márgenes óptimos
5. **Árboles de Decisión**: Modelo interpretable de clasificación
6. **Naive Bayes**: Clasificación probabilística
7. **K-Means**: Segmentación de exportaciones similares

---

**Autor**: Proyecto Machine Learning - Análisis de Exportaciones

**Fecha**: 2025

**Versión**: 1.0